<a href="https://colab.research.google.com/github/jayjinnie/gbdcon/blob/main/%EA%B2%BD%EB%B6%81_%EC%95%84%EC%9D%B4%ED%85%9C_%EA%B8%B0%EB%B0%98_%ED%98%91%EC%97%85_%ED%95%84%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요한 데이터 확인
#### userid: 회원번호
#### mercid : 상품코드
#### merc_name: 상품명
#### rating: 선호도
#### category: 분류명
##### rating 정보를 가진 콘텐츠 테이블과 회원정보 테이블 crosstab으로 merge

In [ ]:
import pandas as pd

In [ ]:
contents = pd.read_excel("콘텐츠 테이블(with rating).xlsx")
contents.head()
# 상품코드, 상품명, 선호도, 분류명이 있음

,상품코드,기본분류,분류명,상품유형,상품명,제조사,판매가격,기본배송비,조회수,판매수량,업체ID,선호도
0,157092753,201030,곶감/반건시/감말랭이,NaN,[상주전통곶감] 전통곶감 건시 4kg(65~70과),상주전통곶감,89000,0,0,0,d3f908bbea5d2d7cac866508dff9b2ffdf0088ab55d178...,22.961649
1,1578008979,201030,곶감/반건시/감말랭이,농산물,[구릿뜰농원] 건시3호 48구,구릿뜰농원,55000,4000,130,0,b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...,19.849017
2,1578031964,201030,곶감/반건시/감말랭이,농산물,[구릿뜰농원] 건시4호 36구,구릿뜰농원,45000,4000,223,1,b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...,18.306281
3,1578034429,201030,곶감/반건시/감말랭이,농산물,[구릿뜰농원] 건시 낱개 포장 30구,구릿뜰농원,40000,4000,141,0,b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...,19.623925
4,1578034546,201030,곶감/반건시/감말랭이,농산물,[구릿뜰농원] 건시 낱개 포장1호,구릿뜰농원,30000,4000,136,0,b4f5eabe0dd4a2502f409f9710682f72bbbc79de195121...,19.725603


In [ ]:
customer = pd.read_excel("회원정보테이블.xlsx", header=1)
customer.head()
# 회원번호가 있음

,회원번호,회원ID,성별,생년월일,기본주소,가입일시,탈퇴일,업체구분,사업자여부
0,1,2c6e78c8162fde1d091cb821f3364c6e04f1473e9217e3...,NaN,NaN,경북 안동시 경동로,2019-09-20 16:05:33,NaN,0,0
1,60097,17ffc7f9bfd26a64642ee64c514a526edd4a0ab6fdbb77...,NaN,NaN,NaN,2013-02-22 00:00:00,NaN,0,0
2,60098,ed260379750ec817c68d8070d4507083645fb4d5fd81ac...,NaN,NaN,NaN,2012-01-13 00:00:00,NaN,0,0
3,60099,2436322996a78dfe1acede3c2d142872b7d5be7af280d6...,NaN,NaN,NaN,2015-04-13 00:00:00,NaN,0,0
4,60100,6e2ad3000b49e2f9d7594f5b3f71d0bdabddf666c44b59...,NaN,55.0,경상북도 포항시 북구,2013-10-09 00:00:00,0000-00-,1,0


In [ ]:
buy = pd.read_excel("장바구니테이블.xlsx", header=1)
# 배송완료 예정일 0000-00-00 제외
# 회원아이디, 상품코드, 상품명이 있음
buy.head()

,장바구니번호,주문번호,회원ID,상품코드,상품명
0,2079385,1291ff3422b02515594f564eae65980860baa03bf55809...,4756031f0b9f62c7c1673a1f9d956005829ab3db0ba9b5...,1577772279,★제503수요특가★안동마(산마) 상품 - 5kg [풍요로운농장]
1,2079386,1291ff3422b02515594f564eae65980860baa03bf55809...,4756031f0b9f62c7c1673a1f9d956005829ab3db0ba9b5...,1577146719,★제503수요특가★건강하게 키운 사과 9kg42과
2,2079400,b3890fcb75c5491fcdeef3e7114716df153001ac6e9ac2...,b38a20d39ec2e7ef5795b6868085a704597586623d4063...,1570787886,[파머story농장] [특가제안] 영주사과 10kg 34~44과 흠집사과 풍기사과
3,2079401,1a6e9b3dcc6aca97f94eaed5f8f47946ee85b7e4b8de0d...,b38a20d39ec2e7ef5795b6868085a704597586623d4063...,1570784838,[GAP인증] 청송꿀사과 부사 10kg (35과 크기-中小)
4,2079404,b58b99a7e8136ad5f4b0a20194d6c3b331b7affd6c9a9f...,adec4f204eb273c57e8b1aac284285892ae59c33b61978...,1577772279,★제503수요특가★안동마(산마) 상품 - 5kg [풍요로운농장]


In [ ]:
# 장바구니테이블과 회원정보 테이블 '회원ID'를 기준으로 merge
# 회원 번호 추출

customers = pd.merge(customer, buy, on="회원ID")
customers = customers[["회원번호", "상품명"]]
customers.head()

,회원번호,상품명
0,1,[풍요로운농장] 감시골 양파즙
1,1,"★설특판★[더파머스] 사과 배 혼합 과일선물세트 5.5kg 사과6, 배6"
2,1,[해도지영농조합] 해도지 2020년 하우스 감자 5kg(택배비무료)
3,1,무농약 연근 쓱결제용
4,1,★원데이★마누카꿀 홍삼절편 오리지널(한문패키지)


In [ ]:
# customers와 contents '상품명'을 기준으로 merge
# 회원번호, 상품명, 선호도 추출
items = pd.merge(customers, contents, on="상품명")
items = items[["회원번호", "상품명", "선호도"]]
items.head()

,회원번호,상품명,선호도
0,1,무농약 연근 쓱결제용,18.680282
1,1,★이웃사촌★ [성주로컬푸드]고소애500g,20.818027
2,1,★이웃사촌★ [성주로컬푸드]고소애500g,20.818027
3,1,[오세창곤충체험농장]파워뱅이분말 180g,18.806200
4,158993,[오세창곤충체험농장]파워뱅이분말 180g,18.806200


# pivot 테이블 구성
#### data: 상품 선호도 rating (based on '조회수', '판매수량')
#### index: 상품명
#### columns: 회원번호

In [ ]:
merc_custom_rating = items.pivot_table('선호도', index='상품명', columns='회원번호') # 아이템 기반 테이블
# 사용자 기반 테이블: custom_merc_rating = items.pivot_table('선호도', index='회원번호', columns='상품명')
merc_custom_rating.head()

회원번호,1,60251,60603,60729,60846,61002,61062,61184,61484,61677,61772,61775,61891,61926,62191,62200,62241,62249,62350,62362,62410,62420,62439,62611,62744,62907,62968,62973,62997,63165,63290,63387,63601,63790,64118,64240,64582,64721,64803,65304,...,169063,169065,169067,169070,169080,169091,169095,169109,169111,169121,169137,169153,169157,169165,169170,169173,169176,169177,169179,169188,169208,169222,169228,169231,169233,169236,169248,169260,169261,169263,169264,169266,169269,169278,169279,169280,169282,169286,169293,169295
상품명,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"""힘내세요, 우리아빠"" 기세움[굼벵이(꽃벵이)진액/엑기스]",16.495395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(경북청년농부들) 민서네버섯농장 건슬라이스표고버섯 150g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(문경약돌돼지) 문경약돌돼지 LA식 갈비(냉동) 1kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(문경약돌돼지) 문경약돌돼지 목살(냉동) 500g*2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(밀웜)고소애 건조 100g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# NaN 데이터에 fillna를 이용해 null 값 채워주기
merc_custom_rating.fillna(0, inplace=True)
merc_custom_rating.head()

회원번호,1,60251,60603,60729,60846,61002,61062,61184,61484,61677,61772,61775,61891,61926,62191,62200,62241,62249,62350,62362,62410,62420,62439,62611,62744,62907,62968,62973,62997,63165,63290,63387,63601,63790,64118,64240,64582,64721,64803,65304,...,169063,169065,169067,169070,169080,169091,169095,169109,169111,169121,169137,169153,169157,169165,169170,169173,169176,169177,169179,169188,169208,169222,169228,169231,169233,169236,169248,169260,169261,169263,169264,169266,169269,169278,169279,169280,169282,169286,169293,169295
상품명,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"""힘내세요, 우리아빠"" 기세움[굼벵이(꽃벵이)진액/엑기스]",16.495395,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(경북청년농부들) 민서네버섯농장 건슬라이스표고버섯 150g,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(문경약돌돼지) 문경약돌돼지 LA식 갈비(냉동) 1kg,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(문경약돌돼지) 문경약돌돼지 목살(냉동) 500g*2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(밀웜)고소애 건조 100g,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 아이템 기반 협업 필터링 (Item based Collaborative Filtering)
#### 유사한 아이템(상품) = '선호도'가 비슷한 아이템을 의미함
#### 현재 선호도가 data로 들어가 있으므로 코사인 유사도 구하기

### 코사인 유사도 구하기

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

item_based_collabor = cosine_similarity(merc_custom_rating)
item_based_collabor
# 아이템 별로 서로 유사한 정보의 값을 가지게 됨
# 자기 자신과 일치하는 값은 1이며, 유사도가 높을수록 1에 가까운 값을 가짐

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
item_based_collabor = pd.DataFrame(data = item_based_collabor, index = merc_custom_rating.index, columns= merc_custom_rating.index)
item_based_collabor

상품명,"""힘내세요, 우리아빠"" 기세움[굼벵이(꽃벵이)진액/엑기스]",(경북청년농부들) 민서네버섯농장 건슬라이스표고버섯 150g,(문경약돌돼지) 문경약돌돼지 LA식 갈비(냉동) 1kg,(문경약돌돼지) 문경약돌돼지 목살(냉동) 500g*2,(밀웜)고소애 건조 100g,(밀웜)고소애 분말 100g,(울진오징어) 배오징어 반건조오징어 10마리 (100% 국산),**곶감타래과자**,100%국내산 맑은 참기름 180ml,100%국내산 맑은 참기름 300ml,100%국내산 볶지 않고 생으로 짜낸 맑은 생들기름180ml,100%국내산 볶지 않고 생으로 짜낸 맑은 생들기름300ml,100%국내산 참.들 기름세트,2+1 마누카꿀 홍삼절편 3종 택1,2020 의성깐마늘(토종) 상 1kg,2020 의성깐마늘(토종) 하 1kg,2020년 경북사과 온라인 홍보행사 부사 3kg 10-12과 / 11월 9일~13일까지 행사,2020년 유기농인증 토종가시복분자(급냉),"2020년 하지감자(중, 4,번) 10kg",2020년 햅쌀 안동밥상 백진주 10kg 백미,2020년 햅쌀 안동밥상 백진주 10kg 현미,2020년 햅쌀_9분도 쌀눈쌀 10kg,2020년 햅쌀_행복한쌀 5kg,2020년 햇 팥 1kg (적두),2020년 햇마늘 향이 진한 의성마늘 3kg,"2020년, 국산 고추가루,햇살담은 고춧가루,1kg",2020년산 햇고추/(주)다산식품 베타글루칸 영양청결고춧가루/고춧가루 1kg/햇고춧가루,"20년 속이 꽉찬 유기농 양파 (대) 3kg, 5kg, 10kg",20년 수확 알싸한 친환경 깐마늘 1kg (대),20년산 햅쌀 1kg 4kg 10kg,20년산 햇 찰보리쌀 4kg,264청포도와인 광야 미디움스위트,264청포도와인 꽃 스위트,GAP인증 청송꿀사과백석탄농원 (사과즙 120ml x 50팩 2박스),[ 농부플러스 ] 국산 냉압착 생들기름 180ml,[ 농부플러스 ] 국산 냉압착 생들기름 240ml,[ 농부플러스 ] 국산 냉압착생들기름 240ml 두병선물세트,[ 농부플러스 ] 국산 저온착유 들기름 180ml,[ 농부플러스 ] 국산 저온착유 들기름 240ml,[ 농부플러스 ] 국산 저온착유 참기름 180ml,...,푸드버킷 착즙 포도즙 100 120팩,푸드버킷 착즙 포도즙 100 30팩,푸드버킷 착즙 포도즙 100 60팩,풋사과 동결건조 파우더,풍기농부홍삼정골드600g+50g,풍기농부홍삼청프리미엄240g,프리미엄 레드 홍삼초코봉봉 24개입,피크닉사과 가정용 흠과 10kg,하니베 현미 누룽지 130g 1+1+1,한고을 한재미나리1kg (무농약),한고을 한재미나리200g (무농약),항아초 2종세트(사과300ml+솔잎300ml),항아초 마시는 식초3종세트(사과300ml+솔잎300ml+현미300ml),항아초 마시는사과초 500ml,항아초 마시는현미초 500ml,항아초 생강식초 300ml,항아초 현미식초 500ml,해방풍,해인들강정,"햇 반태양초 고춧가루 250g ,500g,1000g",햇 영양고추가루/ 생산자직판/ 직접말린태양초/ 영양군 고추/ 고춧가루5근(2.5kg)김장용보통맵기,햇 천도 복숭아즙 과일즙 80ml 1포,햇땅콩 물좋은 예천산 생땅콩 2kg,형제농장 어수리나물 1Kg,"형제농장 어수리나물 500g,1Kg",호밀그대로 스낵,혼합세트 (375mlx2개),혼합세트 (750mlx2개),홍로사과 5KG (14~16과) 선물용,홍잠 100g / 농촌진흥청 특허 기술 홍잠(익힌숙잠),홍잠분말 50g / 누에농장 생산제품,후포리 명품 홍게찜 10마리,"흥림산웰빙산나물 모둠나물,곰취,취나물 1Kg","흥림산웰빙산나물 모둠나물,곰취,취나물 1Kg(최소 구매수량 2Kg)",［거인황금팜］국내산 돼지고기 냉장 등심덧살 500g(1등급이상),［문경몰］무농약 오미자 생과,"［문경몰］문경 순오미자 1L (오미자추출액 100%, 무설탕)",［문경몰］알갱이만쏙 오미자 당절임,［문경몰］오미자 당절임,［문경몰］오미자 생과
상품명,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"""힘내세요, 우리아빠"" 기세움[굼벵이(꽃벵이)진액/엑기스]",1.0,0.0,0.0,0.0,0.0,0.57735,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
(경북청년농부들) 민서네버섯농장 건슬라이스표고버섯 150g,0.0,1.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
(문경약돌돼지) 문경약돌돼지 LA식 갈비(냉동) 1kg,0.0,0.0,1.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.11547,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
(문경약돌돼지) 문경약돌돼지 목살(냉동) 500g*2,0.0,0.0,0.0,1.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
(밀웜)고소애 건조 100g,0.0,0.0,0.0,0.0,1.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0

In [ ]:
# 입력한 상품(상푸명을 입력)과 유사한(코사인 유사도) 5개의 상품을 추천하는 기능의 함수
def get_item_based_collabor(merc_name):
    return item_based_collabor[merc_name].sort_values(ascending=False)[:7]

In [ ]:
# 임의의 상품명 입력
get_item_based_collabor('피크닉사과 가정용 흠과 10kg')

상품명
피크닉사과 가정용 흠과 10kg                                        1.000000
답운농장 큰며느리가 만든 현미누룽지, 흑임자[검정깨]누룽지,새싹보리누룽지,홍삼누룽지,천년초누룽지    0.169031
[하늘기둥농장] 간편 소포장 청국장 2킬로(200그램*10)                        0.169031
명품안동소주 담금주 3600ml 30도                                    0.169031
★이웃사촌★[달산마을]햇 알호두(30알)380g~400g                          0.119523
[윤푸드] 이로우니 우리쌀 연근돈까스 650g (130g*5개)                      0.119523
[대우농장] 완숙 토마토 5kg (특품)                                   0.119523
Name: 피크닉사과 가정용 흠과 10kg, dtype: float64

### if '피크닉사과 가정용 흠과 10kg'를 구매한 고객에게 해당 상품과 **유사한 선호도**(판매수량, 조회수 기반)를 가진 상품 추천 목록을 보여줌
#### 선호도는 판매수량과 조회수를 기반으로 계산되었으나, 판매수량과 조회수가 '높은' 상품만 추천하는 것은 아니므로 의미가 있음, 결과적으로 추천 시스템을 통해 보다 다양한 상품을 맞춤형으로 추천하는 것이 가능함 (그렇지만 고객으로부터 상품별 평점 데이터를 추가적으로 수집한다면, 추천 알고리즘의 기능성을 개선/보완할 수 있음)